<a href="https://colab.research.google.com/github/gershom1/Practical-Introduction-to-python/blob/patch-1/Copy_of_CapsNet_LungsCancer_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.layers import Input, Conv2D, Dense
from tensorflow.keras.layers import Reshape, Layer, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
import numpy as np
import tensorflow as tf
import cv2
import os
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pylab as plt
import cv2   
import os
from google.colab.patches import cv2_imshow # for image display
import pandas as pd

In [ ]:
# mount the google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df_train = pd.read_csv('/content/drive/My Drive/GershomProject/label.csv') # read the dataset to a pandas data frame
df_train.head() # check out the details of the dataset

,image_id,class
0,00000.jpg,0
1,00001.jpg,0
2,00002.jpg,0
3,00003.jpg,0
4,00004.jpg,0


In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array
from tqdm import tqdm

def load_labels(inputPath):
    # initialize the list of column names in the CSV file and then load it using Pandas
    col_list = ["image_id", "class"]
    df = pd.read_csv(inputPath, usecols=col_list)
    return df

# this function is for the sort function in load_gaze_images() function below
def getint(name):
    basename, _ = name.split('.')
    return int(basename)

def read_image(fname, IMAGE_DIMS):
    # read each image in colored
    img = cv2.imread(fname) 
    
    # resize each image
    f_h, f_w = int(IMAGE_DIMS[0]), int(IMAGE_DIMS[1]) # height and width
    img = cv2.resize(img, (f_h, f_w))
    
    # convert image to RGB for ease of training
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img_to_array(img)

    return img

# process and dataset
def load_imageAndLabel(dir, IMAGE_DIMS, dataset_name, label_file):
    batch_data = []
    classes = []
    
    # get the path for the labels. 
    label_path = os.path.join(dir, label_file)
    df_label = load_labels(label_path)
    
    # get the path for all the files in the current sub directory
    img_path = os.path.join(dir, dataset_name) 
    
    # get the file names for all the files
    files = os.listdir(img_path)

    n_img = len(files)  # get the total number of images
    for i in tqdm(range(n_img)): 
        # get the file path 
        fname = os.path.join(img_path, files[i])
        print(fname)
        idx = df_label[df_label["image_id"] == files[i]].index.values # get the index of the current image
        clas = df_label["class"].values[idx]
        classes.append(clas)
        img = read_image(fname, IMAGE_DIMS)
        batch_data.append(img) # create batch data
    one = classes.count(0)
    two = classes.count(1)
    three = classes.count(2)
    norm = classes.count(3)
    return np.array(batch_data), np.array(classes)

In [ ]:
# Load the dataset
img_dir = r'/content/drive/My Drive/GershomProject/LungsCancerDataset/'
files = os.listdir(img_dir)

In [ ]:
input_shape = [224, 224, 3]
dimA, dimB, n_channel = input_shape
tot_n_pixels = dimA * dimB * n_channel
train_dir = '/content/drive/My Drive/GershomProject/'
#img_dir = r'/content/drive/My Drive/GershomProject/LungsDataset/'
dataset_name = "LungsCancerDataset" 
label_file = "label.csv"
n_class = 4
n_routing = 3

print("[INFO] loading training and test data...")
images, labels = load_imageAndLabel(train_dir, input_shape, dataset_name, label_file)

[INFO] loading training and test data...


  0%|          | 0/4 [00:00<?, ?it/s]

/content/drive/My Drive/GershomProject/LungsCancerDataset/normal


error: ignored

In [ ]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
split = train_test_split(images, labels, test_size=0.2, random_state=42)
(x_train, x_test, y_train, y_test) = split

In [ ]:
# Scale or normalize the dataset
print("[INFO] scaling the dataset to range [0,1]...")
x_train = x_train.reshape(-1, dimA, dimB, n_channel).astype('float32') / 255.0 # reshape and normalize the training set
x_test = x_test.reshape(-1, dimA, dimB, n_channel).astype('float32') / 255.0 # reshape and normalize the test set

# one hot encode target values
y_train = to_categorical(y_train.astype('float32'))
y_test = to_categorical(y_test.astype('float32'))

In [ ]:
# join the train and test dataset
X = np.concatenate((x_train, x_test), axis=0)
Y = np.concatenate((y_train, y_test), axis=0)

In [ ]:
# pass the reshaped output through a non-linear squash function to ensure that the length of each vector is between 0 and 1
def squash(output_vector, axis=-1):
    # take norm of input vectors
    norm = tf.reduce_sum(tf.square(output_vector), axis, keepdims=True) 

    # use the formula for non-linear function to return squashed output
    return output_vector * norm / ((1 + norm) * tf.sqrt(norm + 1.0e-10))

In [ ]:
# input from the digit capsule is masked with the original or actual label. This is done to ensure that the decoder is trained on the correct digit capsule.
class MaskingLayer(Layer):
    def call(self, inputs, **kwargs):
        input, mask = inputs
        return K.batch_dot(input, mask, 1)

    def compute_output_shape(self, input_shape):
        *_, output_shape = input_shape[0]
        return (None, output_shape)

In [ ]:
# the primary capsule layer performs the following: conv, reshape, and lambda
def PrimaryCapsule(n_vector, n_channel, n_kernel_size, n_stride, padding='valid'):
    def builder(inputs):
        output = Conv2D(filters=n_vector * n_channel, kernel_size=n_kernel_size, strides=n_stride, padding=padding)(inputs)
        output = Reshape( target_shape=[-1, n_vector], name='primary_capsule_reshape')(output) # reshape to the required number of capsules and vector per capsule
        return Lambda(squash, name='primary_capsule_squash')(output) # squash the reshaped output to make length of vector b/w 0 and 1
    return builder

In [ ]:
class DigitCapsuleLayer(Layer):
    def __init__(self, n_capsule, n_vec, n_routing, **kwargs):
        super(DigitCapsuleLayer, self).__init__(**kwargs)
        self.n_capsule = n_capsule
        self.n_vector = n_vec
        self.n_routing = n_routing
        self.kernel_initializer = initializers.get('he_normal')
        self.bias_initializer = initializers.get('zeros')

    def build(self, input_shape): # input_shape is a 4D tensor
        _, self.input_n_capsule, self.input_n_vector, *_ = input_shape
        #self.W = self.add_weight(shape=[self.input_n_capsule, self.n_capsule, self.input_n_vector, self.n_vector], initializer=self.kernel_initializer, name='W')
        #self.W = self.add_weight(shape=[self.n_capsule, self.input_n_capsule, self.input_n_vector, self.n_vector], initializer=self.kernel_initializer, name='W')
        self.W = self.add_weight(shape=[self.n_capsule, self.input_n_capsule, self.n_vector, self.input_n_vector], initializer=self.kernel_initializer, name='W')
        self.bias = self.add_weight(shape=[1, self.input_n_capsule, self.n_capsule, 1, 1], initializer=self.bias_initializer, name='bias', trainable=False)
        self.built = True

    def call(self, inputs, training=None):
        # Expand the input in axis=1, tile in that axis to num_capsule, and 
        # expands another axis at the end to prepare the multiplication with W.
        #  inputs.shape=[None, input_num_capsule, input_dim_capsule]
        #  inputs_expand.shape=[None, 1, input_num_capsule, input_dim_capsule]
        #  inputs_tiled.shape=[None, num_capsule, input_num_capsule, 
        #                            input_dim_capsule, 1]
        inputs_expand = tf.expand_dims(inputs, 1)
        inputs_tiled  = tf.tile(inputs_expand, [1, self.n_capsule, 1, 1])
        inputs_tiled  = tf.expand_dims(inputs_tiled, 4)

        # Compute `W * inputs` by scanning inputs_tiled on dimension 0 (map_fn).
        # - Use matmul (without transposing any element). Note the order!
        # Thus:
        #  x.shape=[num_capsule, input_num_capsule, input_dim_capsule, 1]
        #  W.shape=[num_capsule, input_num_capsule, dim_capsule,input_dim_capsule]
        # Regard the first two dimensions as `batch` dimension,
        # then matmul: [dim_capsule, input_dim_capsule] x [input_dim_capsule, 1]-> 
        #              [dim_capsule, 1].
        #  inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule, 1]
        
        inputs_hat = tf.map_fn(lambda x: tf.matmul(self.W, x), elems=inputs_tiled)     

        # Begin: Routing algorithm ----------------------------------------------#
        # The prior for coupling coefficient, initialized as zeros.
        #  b.shape = [None, self.num_capsule, self.input_num_capsule, 1, 1].
        b = tf.zeros(shape=[tf.shape(inputs_hat)[0], self.n_capsule, 
                            self.input_n_capsule, 1, 1])

        assert self.n_routing > 0, 'The routings should be > 0.'
        for i in range(self.n_routing):
            # Apply softmax to the axis with `num_capsule`
            #  c.shape=[batch_size, num_capsule, input_num_capsule, 1, 1]
            c = layers.Softmax(axis=1)(b)

            # Compute the weighted sum of all the predicted output vectors.
            #  c.shape =  [batch_size, num_capsule, input_num_capsule, 1, 1]
            #  inputs_hat.shape=[None, num_capsule, input_num_capsule,dim_capsule,1]
            # The function `multiply` will broadcast axis=3 in c to dim_capsule.
            #  outputs.shape=[None, num_capsule, input_num_capsule, dim_capsule, 1]
            # Then sum along the input_num_capsule
            #  outputs.shape=[None, num_capsule, 1, dim_capsule, 1]
            # Then apply squash along the dim_capsule
            outputs = tf.multiply(c, inputs_hat)
            outputs = tf.reduce_sum(outputs, axis=2, keepdims=True)
            outputs = squash(outputs, axis=-2)  # [None, 10, 1, 16, 1]

            if i < self.n_routing - 1:
                # Update the prior b.
                #  outputs.shape =  [None, num_capsule, 1, dim_capsule, 1]
                #  inputs_hat.shape=[None,num_capsule,input_num_capsule,dim_capsule,1]
                # Multiply the outputs with the weighted_inputs (inputs_hat) and add  
                # it to the prior b.  
                outputs_tiled = tf.tile(outputs, [1, 1, self.input_n_capsule, 1, 1])
                agreement = tf.matmul(inputs_hat, outputs_tiled, transpose_a=True)
                b = tf.add(b, agreement)

        # End: Routing algorithm ------------------------------------------------#
        # Squeeze the outputs to remove useless axis:
        #  From  --> outputs.shape=[None, num_capsule, 1, dim_capsule, 1]
        #  To    --> outputs.shape=[None, num_capsule,    dim_capsule]
        outputs = tf.squeeze(outputs, [2, 4])
        return outputs

    def compute_output_shape(self, input_shape):
        # output current layer capsules
        return (None, self.n_capsule, self.n_vector)

In [ ]:
class LengthLayer(Layer):
    def call(self, inputs, **kwargs):
        return tf.sqrt(tf.reduce_sum(tf.square(inputs), axis=-1, keepdims=False))

    def compute_output_shape(self, input_shape):
        *output_shape, _ = input_shape
        return tuple(output_shape)

In [ ]:
# probabilistic loss function used for classifying digits image.
def margin_loss(y_ground_truth, y_prediction):
    _m_plus = 0.9
    _m_minus = 0.1
    _lambda = 0.5
    L = y_ground_truth * tf.square(tf.maximum(0., _m_plus - y_prediction)) + _lambda * ( 1 - y_ground_truth) * tf.square(tf.maximum(0., y_prediction - _m_minus))
    return tf.reduce_mean(tf.reduce_sum(L, axis=1))

In [ ]:
# encoder network
x = Input(shape=input_shape) # input for the encoder network
conv1 = Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)
primary_capsule = PrimaryCapsule(n_vector=8, n_channel=32, n_kernel_size=9, n_stride=2)(conv1)
digit_capsule = DigitCapsuleLayer(n_capsule=n_class, n_vec=16, n_routing=n_routing, name='digit_capsule')(primary_capsule)
output_capsule = LengthLayer(name='output_capsule')(digit_capsule)

In [ ]:
# decoder network to reconstruct the input image
mask_input = Input(shape=(n_class, )) # input for the digits capsule layer
mask = MaskingLayer()([digit_capsule, mask_input])  # two inputs - the digit capsule and the masked input
dec = Dense(512, activation='relu')(mask)
dec = Dense(1024, activation='relu')(dec)
dec = Dense(tot_n_pixels, activation='sigmoid')(dec) # reconstruct the input image
dec = Reshape(input_shape)(dec)

In [ ]:
# compile the model
model = Model([x, mask_input], [output_capsule, dec])
model.compile(optimizer='adam', loss=[ margin_loss, 'mae' ], metrics=[ margin_loss, 'mae', 'accuracy'])

In [ ]:
# train the model
model.fit([X, Y], [Y, X], batch_size=128, epochs=3, validation_split=0.2)

In [ ]:
# evaluate the model
label_predicted, image_predicted = model.predict([x_test, y_test])

In [ ]:
# convert the predicted label to categorical values so that we can calculate the performance metrics
pred_categorical = []
actual_categorical = []  
for i in range(0, len(label_predicted)):
  pred_categorical.append(np.argmax(label_predicted[i])) 
  actual_categorical.append(np.argmax(y_test[i]))

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(actual_categorical, pred_categorical)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(actual_categorical, pred_categorical, average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(actual_categorical, pred_categorical, average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(actual_categorical, pred_categorical, average='micro')
print('F1 score: %f' % f1)
# kappa
kappa = cohen_kappa_score(actual_categorical, pred_categorical)
print('Cohens kappa: %f' % kappa)

In [ ]:
# confusion matrix
matrix = confusion_matrix(actual_categorical, pred_categorical)
print(matrix)

In [ ]:
# visualize the predicted images

n_samples = 5
plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    sample_image = x_test[index].reshape(28, 28)
    plt.imshow(sample_image, cmap="binary")
    plt.title("Label:" + str(y_test[index]))
    plt.axis("off")

plt.show()

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    sample_image = image_predicted[index].reshape(28, 28)
    plt.imshow(sample_image, cmap="binary")
    plt.title("Predicted:" + str(np.argmax(label_predicted[index])))
    plt.axis("off")

plt.show()